In [ ]:
from pathlib import Path
import sys, math, time

import torch
from torch.utils.data import DataLoader

CODE_DIR = Path("/kaggle/input/transformer-iwslt15/src-code/src") 
DATA_ROOT = Path("/kaggle/input/transformer-iwslt15/data-en-vi")
PROCESSED_DIR = DATA_ROOT / "processed"
SPM_MODEL = DATA_ROOT / "spm" / "spm_unigram.model"
print("CODE_DIR:", CODE_DIR)
print("PROCESSED_DIR:", PROCESSED_DIR)
print("SPM_MODEL:", SPM_MODEL)

sys.path.append(str(CODE_DIR))

from tokenizer import SubwordTokenizer
from dataset import NMTDataset, collate_fn
from model import Transformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


In [ ]:
# Khởi tạo tokenizer từ SentencePiece model
tokenizer = SubwordTokenizer(str(SPM_MODEL))

vocab_size = tokenizer.sp.vocab_size()
pad_id = tokenizer.pad_id
bos_id = tokenizer.bos_id
eos_id = tokenizer.eos_id

print("Vocab size:", vocab_size)
print("pad/bos/eos:", pad_id, bos_id, eos_id)

MAX_SRC_LEN = 70
MAX_TGT_LEN = 70
BATCH_SIZE = 64  

# Tạo Dataset
train_dataset = NMTDataset(
    data_dir=str(PROCESSED_DIR),
    split="train",
    tokenizer=tokenizer,
    max_src_len=MAX_SRC_LEN,
    max_tgt_len=MAX_TGT_LEN,
)

valid_dataset = NMTDataset(
    data_dir=str(PROCESSED_DIR),
    split="valid",
    tokenizer=tokenizer,
    max_src_len=MAX_SRC_LEN,
    max_tgt_len=MAX_TGT_LEN,
)

print("Train size:", len(train_dataset))
print("Valid size:", len(valid_dataset))

# Tạo DataLoader
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=lambda batch: collate_fn(batch, pad_id=pad_id),
)

valid_loader = DataLoader(
    valid_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=lambda batch: collate_fn(batch, pad_id=pad_id),
)

batch = next(iter(train_loader))
for k, v in batch.items():
    print(k, v.shape, v.dtype)


In [ ]:
import os, math, time
import torch
import torch.nn as nn
import torch.optim as optim

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🚀 Device:", DEVICE)

# 1.Hyperparameters
D_MODEL = 384
N_LAYERS = 4
N_HEADS  = 8
D_FF     = 1536
DROPOUT  = 0.1
MAX_LEN  = 5000

N_EPOCHS = 20
WARMUP_STEPS = 4000
LABEL_SMOOTHING = 0.1

configs = [
    {
        "prefix": "baseline",  
        "name": "BASELINE (Post-Norm + ReLU)",
        "activation": "relu",
        "norm_first": False,
    },
    {
        "prefix": "modern",   
        "name": "MODERN (Pre-Norm + GeLU)",
        "activation": "gelu",
        "norm_first": True,
    }
]

# 2.HELPERS

def make_masks(src, tgt_in, pad_id):
    # src_mask: [B, 1, 1, S]
    src_mask = (src != pad_id).unsqueeze(1).unsqueeze(2)

    # tgt_mask: [B, 1, T, T]
    tgt_pad_mask = (tgt_in != pad_id).unsqueeze(1).unsqueeze(2)
    T = tgt_in.size(1)
    nopeak_mask = torch.tril(torch.ones((1, 1, T, T), device=src.device)).bool()

    tgt_mask = tgt_pad_mask & nopeak_mask
    return src_mask, tgt_mask

class NoamLR:
    def __init__(self, optimizer, d_model, warmup_steps=4000, factor=1.0):
        self.optimizer = optimizer
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        self.factor = factor
        self.step_num = 0

    def rate(self):
        step = max(self.step_num, 1)
        return self.factor * (self.d_model ** -0.5) * min(step ** -0.5, step * (self.warmup_steps ** -1.5))

    def step(self):
        self.step_num += 1
        lr = self.rate()
        for g in self.optimizer.param_groups:
            g["lr"] = lr

def run_epoch(model, dataloader, optimizer, criterion, device, pad_id, is_train=True, scheduler=None):
    model.train() if is_train else model.eval()
    total_loss = 0.0
    total_tokens = 0

    for batch in dataloader:
        src = batch["src_ids"].to(device)
        tgt_in = batch["tgt_in_ids"].to(device)
        tgt_out = batch["tgt_out_ids"].to(device)

        src_mask, tgt_mask = make_masks(src, tgt_in, pad_id)

        if is_train:
            optimizer.zero_grad()

        with torch.set_grad_enabled(is_train):
            # Model forward
            logits = model(src, tgt_in, src_mask, tgt_mask)  # [B, T, V]
            V = logits.size(-1)

            loss = criterion(logits.view(-1, V), tgt_out.view(-1))

            if is_train:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                if scheduler is not None:
                    scheduler.step()

        # Tính loss trung bình theo token
        non_pad = (tgt_out != pad_id).sum().item()
        total_loss += loss.item() * non_pad
        total_tokens += non_pad

    avg_loss = total_loss / max(total_tokens, 1)
    ppl = math.exp(avg_loss) if avg_loss < 20 else float("inf")
    return avg_loss, ppl

In [ ]:
import matplotlib.pyplot as plt

# 3. TRAINING LOOP
def train_pipeline(config, train_loader, valid_loader):
    print(f"\n{'='*60}")
    print(f"🚀 START TRAINING: {config['name']}")
    print(f"   Config: Activation={config['activation']} | Pre-Norm={config['norm_first']}")
    print(f"{'='*60}")
    save_name = f"{config['prefix']}_L{N_LAYERS}_H{N_HEADS}_D{D_MODEL}.pt"
    save_path = f"/kaggle/working/{save_name}"
    config["save_path"] = save_path
    # 1. Khởi tạo Model
    model = Transformer(
        src_vocab_size=vocab_size, tgt_vocab_size=vocab_size,
        d_model=D_MODEL, n_layers=N_LAYERS, n_heads=N_HEADS, 
        d_ff=D_FF, dropout=DROPOUT, max_len=MAX_LEN,
        activation=config["activation"], 
        norm_first=config["norm_first"]
    ).to(DEVICE)
    
    # 2. Setup Optimizer
    criterion = nn.CrossEntropyLoss(ignore_index=pad_id, label_smoothing=LABEL_SMOOTHING)
    optimizer = optim.Adam(model.parameters(), lr=0.0, betas=(0.9, 0.98), eps=1e-9)
    scheduler = NoamLR(optimizer, d_model=D_MODEL, warmup_steps=WARMUP_STEPS, factor=1.0)
    
    best_valid_loss = float("inf")
    
    # Tạo dictionary để lưu lịch sử Loss ---
    history = {
        "train_loss": [],
        "valid_loss": []
    }
    
    # 3. Training Loop
    for epoch in range(1, N_EPOCHS + 1):
        start = time.time()

        train_loss, train_ppl = run_epoch(model, train_loader, optimizer, criterion, DEVICE, pad_id, is_train=True, scheduler=scheduler)
        valid_loss, valid_ppl = run_epoch(model, valid_loader, optimizer, criterion, DEVICE, pad_id, is_train=False, scheduler=None)

        history["train_loss"].append(train_loss)
        history["valid_loss"].append(valid_loss)

        mins = (time.time() - start) / 60
        lr_now = optimizer.param_groups[0]["lr"]
        print(f"Epoch {epoch:02d} | Time: {mins:.2f}m | TrLoss: {train_loss:.4f} | ValLoss: {valid_loss:.4f}")

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), config["save_path"])
            
    print(f"🏁 Xong {config['name']}\n")
    return model, history  

# 4. CHẠY & VẼ BIỂU ĐỒ SO SÁNH
all_histories = {}

# 1. Chạy Training
for conf in configs:
    _, hist = train_pipeline(conf, train_loader, valid_loader)
    all_histories[conf["name"]] = hist

plt.figure(figsize=(15, 6))

# --- Biểu đồ 1: Train Loss ---
plt.subplot(1, 2, 1)
for name, hist in all_histories.items():
    plt.plot(hist["train_loss"], label=name, linewidth=2)
plt.title("Tốc độ hội tụ (Train Loss)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# --- Biểu đồ 2: Validation Loss ---
plt.subplot(1, 2, 2)
for name, hist in all_histories.items():
    plt.plot(hist["valid_loss"], label=name, linewidth=2)
plt.title("Khả năng tổng quát hóa (Valid Loss)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

In [ ]:
import os
print(os.listdir("/kaggle/working/"))